In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName('Python Spark SQL basic example')\
        .config('spark.some.config.option', 'some-value')\
        .getOrCreate()

### Create json file using spark
# SparkContext로 객체 생성
sc = spark.sparkContext

## 데이터프레임 생성하기

### JSON 데이터 생성하기

In [6]:
stringJSONRDD = sc.parallelize((
    """{"id": "123", "name": "Katie", "age": 19, "eyeColor": "brown"}""",
    """{"id": "234", "name": "Michael", "age": 22, "eyeColor": "green"}""",
    """{"id": "345", "name": "Simone", "age": 23, "eyeColor": "blue"}"""
))

### 데이터프레임 생성하기

In [7]:
swimmersJSON = spark.read.json(stringJSONRDD)

### 임시테이블 생성하기

In [8]:
swimmersJSON.createOrReplaceTempView("swimmersJSON")

많은 RDD 함수는 액션 함수가 실행되기 이전까지는 실행되지 않은 트랜스포메이션이다. <br>
예를 들어, 다음 코드에서 sc.parallelize는 spark.read.json을 사용해 RDD를 데이터프레임으로 변환할 때 실행되는 트랜스포메이션이다.

## 간단한 데이터프레임 쿼리

### 데이터프레임 API 쿼리

In [9]:
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



### SQL 쿼리

In [12]:
spark.sql("select * from swimmersJSON").collect()

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

collect() 함수나 show() 함수는 데이터프레임과 SQL 쿼리에 대해 사용할 수 있다. 단, collect() 함수는 데이터프레임의 모든 행을 리턴하고 실행 노드에서 드라이버 노드로 이동하기 때문에 작은 데이터프레임에 대해 사용하는 것이 좋다. 

In [13]:
# 스키마 출력하기
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



## 프로그래밍하는 것처럼 스키마 명시하기

In [14]:
# types import
from pyspark.sql.types import *

# 콤마로 분리된 데이터 생성
stringCSVRDD = sc.parallelize([
    (123, 'Katie', 19, 'brown'),
    (234, 'Micheal', 22, 'green'),
    (345, 'Simone', 23, 'blue')
])

In [15]:
# 스키마를 명시한다.
schema = StructType([
    StructField("id", LongType(), True),
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])

In [16]:
# RDD에 스키마를 적용하고 데이터프레임을 생성
swimmers = spark.createDataFrame(stringCSVRDD, schema)

# 데이터프레임을 이용해 임시 뷰를 생성
swimmers.createOrReplaceTempView("swimmers")

In [17]:
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



## 데이터프레임 API로 쿼리하기

### 행의 개수

In [18]:
swimmers.count()

3

### 필터문 실행하기

In [19]:
# age가 22인 데이터의 id와 age를 출력
swimmers.select("id", "age").filter("age = 22").show()

# 위의 코드를 작성하는 다른 방법
swimmers.select(swimmers.id, swimmers.age).filter(swimmers.age == 22).show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [20]:
# eyeColor가 b로 시작하는 데이터의 name, eyeColor 컬럼을 출력
swimmers.select('name', 'eyeColor').filter("eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



## SQL로 쿼리하기

### 행의 개수

In [23]:
spark.sql("select count(1) from swimmers").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



### 필터문을 where 절을 사용해 실행하기

In [24]:
# age가 22인 데이터의 id와 age를 출력
spark.sql("select id, age from swimmers where age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [25]:
# eyeColor가 b로 시작하는 데이터의 name, eyeColor 컬럼을 출력
spark.sql("select name, eyeColor from swimmers where eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



## 데이터프레임 시나리오: 비행 기록 성능
데이터프레임으로 쿼리할 수 있는 타입을 보여주기 위해 비행 기록 성능에 대한 유스케이스를 살펴보자. 항공사의 지연율과 비행 지연의 원인에 대해 분석해보자. 또한 비행 지연의 여러 변수들을 살펴보기 위해 공항 데이터셋과 조인해보자. 이를 통해 비행 지연과 관련된 변수들을 더 잘 이해할 수 있을 것이다.

### 출발지 데이터셋 준비하기

In [27]:
# path
flightPerFilePath = "./data/departuredelays.csv"
airportsFilePath = "./data/airport-codes-na.txt"

# dataset load
airports = spark.read.csv(airportsFilePath, header='true', inferSchema='true', sep='\t')
airports.createOrReplaceTempView("airports")

# datasest load
flightPerf = spark.read.csv(flightPerFilePath, header='true')
flightPerf.createOrReplaceTempView("FlightPerformance")

# 출발지 지연 데이터셋 캐시 (바로 다음 쿼리가 더 빨리 수행되도록)
flightPerf.cache()

DataFrame[date: string, delay: string, distance: string, origin: string, destination: string]

### 비행 성능 데이터셋과 공항 데이터셋 조인하기

In [28]:
# 도시와 출발지 코드에 따라서 비행 지연 합을 쿼리하기(워싱턴 주에 대해)
spark.sql("""
    select a.City,
    f.origin,
    sum(f.delay) as Delays
    from FlightPerformance f
    join airports a
    on a.IATA = f.origin
    where a.State = "WA"
    group by a.City, f.origin
    order by sum(f.delay) desc""").show()

+-------+------+--------+
|   City|origin|  Delays|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+



In [38]:
# 도시와 출발지 코드에 따라 비행 지연 합을 쿼리하기(워싱턴 주에 대해)
spark.sql("""
select a.City, f.origin, sum(f.delay) as Delays
from FlightPerformance f
    join airports a 
        on a.IATA = f.origin 
where a.State = 'WA'
group by a.City, f.origin
order by sum(f.delay) desc""").show()

+-------+------+--------+
|   City|origin|  Delays|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+



In [39]:
spark.sql("""
select a.State, sum(f.delay) as Delays
    from FlightPerformance f
        join airports a
            on a.IATA = f.origin
where a.Country = 'USA'
group by a.State""").show()

+-----+---------+
|State|   Delays|
+-----+---------+
|   SC|  80666.0|
|   AZ| 401793.0|
|   LA| 199136.0|
|   MN| 256811.0|
|   NJ| 452791.0|
|   OR| 109333.0|
|   VA|  98016.0|
| null| 397237.0|
|   RI|  30760.0|
|   WY|  15365.0|
|   KY|  61156.0|
|   NH|  20474.0|
|   MI| 366486.0|
|   NV| 474208.0|
|   WI| 152311.0|
|   ID|  22932.0|
|   CA|1891919.0|
|   CT|  54662.0|
|   NE|  59376.0|
|   MT|  19271.0|
+-----+---------+
only showing top 20 rows

